# My hangman file

In [1]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections

In [2]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


In [3]:
random.choice(full_dict)

'pwb'

So, I think the best way to do this would be to make a decision tree from scratch, but I think there is an issue of outliers that I am not super sure how to deal with. I think the dictionary is big enough that there are some words that the algorithm should figure out how to throw out. Like if there are 2 moves left, it should make the highest probability assumption it can which eliminates trees with more than 2 moves of surprise.

So... I think how the algorithm should work is, it doesnt just pick the ONE letter that would maximize the entropy next, it should find like a PATH of guesses that maximally reduces the entropy.

I am assuming there are always 6 tries, which seems a bit low...

It seems a bit complex though... If I build a tree I don't have to constantly re-calculate it, but I just move from node to node.

HOW DO I MAKE A DECISION TREE CLASSIFIER WITH NO LABELS THOUGH?

For testing purposes, I want to just remake hangman here so I can mess with it. So gut all of the connection to some external anything. I think I know how it works from running it a bit.

In [4]:
from nltk import ngrams, FreqDist

In [5]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list("a", "e", "i", "o", "u")
        self.guess_list = []


        #set up n-grams for current dictionary
        grams2 = []
        grams3 = []
        grams4 = []
        grams5 = []
        for word in self.full_dictionary:
            #is the word clean? It should be...
            #when I loaded the data it's fine...
            #the API has be a bit nervous so lets just put this here
            word = word.replace(" ", "") #remove any padding that may mess with this

            grams2.extend(list(ngrams(word, 2, pad_left = True, pad_right = True)))
            grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
            grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
            grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))
        
        #Ok, so now need the frequency distributions for these
        freq_dist_2 = FreqDist(grams2)
        freq_dist_3 = FreqDist(grams3)
        freq_dist_4 = FreqDist(grams4)
        freq_dist_5 = FreqDist(grams5)

        #need to actually extract the data from these distributions to use it
        self.freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
        self.freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
        self.freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
        self.freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

        self.words_by_length = {}
        min_word_length = 3 #shortest in the full_dict at least
        max_word_length = 40 #I guess this is pretty arbitrary
        for i in range(min_word_length, max_word_length):
            self.words_by_length.update({i : []})

        #I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
        for word in self.full_dictionary:
            self.full_dictionary[len(word)].append(word)
        
            
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)




    #unfortunately, it looks like there isn't a good way as far as I can tell to avoid a lot of work
    #actually applying the n-grams to the hidden word, its a case-by-case basis of checking letters up to length five
    #ok... so here it is...
        
    #I only want/need to apply n-grams after we have already made at least one guess
    #So, 
    def apply_ngrams(self, index, guess_list, word):
        #Note: I am assuming that word[index] == "_"

        #So current assumption is just "_"
        #we need more 








        #Ok, iterate through all possible n-grams and see which potentially match the word
        return

        

    
    
    
    
    
    
    
    
    
    


    


    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        clean_word = word.replace("_",".") #I am using regex matching to get all possible ngrams that match a word
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                print("Success! the word was: %s" % word)
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            print("You Lose, the answer was: %s" % self.correct_word)
                



In [6]:
tempdict = {1 : ["a"]}

tempdict[1].append("b")
print(tempdict)

{1: ['a', 'b']}


In [7]:
word = "_____"
correctWord = "apple"
letter = "z"
#I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
indices = [i for i, x in enumerate(list(correctWord)) if x == letter]
wordList = list(word)
for i in indices:
    wordList[i] = letter #now I replace the occurrences of

word = "".join(wordList)

#while lidx != -1:
#    word[lidx] = letter
#    lidx = correctWord.find(letter)
print(word)

_____


In [8]:
game = MyHangman()
game.start_game()

TypeError: list expected at most 1 argument, got 5

Ugh, after looking at this, I need to modify the MyHangman code to get some data from it right?

Ok... finally I can start writing the strategy... that shouldn't be too difficult right (uhhh)



# New Approach

# USE NGRAMS

So, it looks like the approach is: take each word and break it down into n-grams. That is, look at each combination of 2,3,4,5 letters in each word and add them to a big list.

Then, find the frequency of each COMBINATION of letters by gram 2,3,4,5

In [48]:
temp = []
_2gram = []
_3gram = []
_4gram = []    
_5gram = []

for word in full_dict:
    temp.append(list(word))         
    _2gram.extend(list(ngrams(word, 2, pad_left=True, pad_right=True)))
    _3gram.extend(list(ngrams(word, 3, pad_left=True, pad_right=True)))
    _4gram.extend(list(ngrams(word, 4, pad_left=True, pad_right=True)))
    _5gram.extend(list(ngrams(word, 5, pad_left=True, pad_right=True)))



from nltk import FreqDist
freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)
print(_2gram[0])

(None, 'a')


In [10]:
print(_2gram[0])

(None, 'a')


In [11]:
temp_list = []
temp_list.extend(list(ngrams("apple", 2, pad_left=True, pad_right=True)))
temp_list.extend(list(ngrams("leafs", 2, pad_left=True, pad_right=True)))
print(temp_list)

[(None, 'a'), ('a', 'p'), ('p', 'p'), ('p', 'l'), ('l', 'e'), ('e', None), (None, 'l'), ('l', 'e'), ('e', 'a'), ('a', 'f'), ('f', 's'), ('s', None)]


In [12]:

from nltk import FreqDist
freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)

In [13]:
freq_2_detached = [(item, freq_2.get(item)) for item in freq_2]


In [14]:
freq_2_detached

[(('s', None), 41635),
 (('e', 'r'), 39200),
 (('i', 'n'), 37762),
 (('e', None), 33665),
 (('t', 'i'), 27596),
 (('o', 'n'), 27389),
 (('e', 's'), 27128),
 (('t', 'e'), 26219),
 (('a', 'n'), 25117),
 (('r', 'e'), 24800),
 ((None, 's'), 24643),
 (('a', 't'), 23611),
 (('a', 'l'), 23550),
 (('e', 'n'), 23199),
 (('d', None), 23114),
 (('e', 'd'), 23098),
 (('l', 'e'), 22135),
 (('r', 'i'), 20879),
 (('i', 's'), 20485),
 (('r', 'a'), 20481),
 (('y', None), 20367),
 (('i', 'c'), 20223),
 (('s', 't'), 20183),
 ((None, 'p'), 19955),
 (('a', 'r'), 19848),
 (('n', 'e'), 19584),
 (('n', 'g'), 19217),
 (('l', 'i'), 19202),
 ((None, 'c'), 18873),
 (('r', 'o'), 18733),
 (('o', 'r'), 17544),
 (('n', None), 17260),
 (('n', 't'), 16960),
 (('l', 'a'), 15234),
 (('u', 'n'), 15154),
 ((None, 'a'), 15016),
 (('i', 't'), 14248),
 (('c', 'o'), 14169),
 (('e', 'l'), 14123),
 (('d', 'e'), 13954),
 (('s', 'e'), 13813),
 (('l', 'l'), 13583),
 (('c', 'a'), 13089),
 (('g', None), 12958),
 (('n', 'i'), 12951),


In [15]:
freq_4

FreqDist({('s', None, None, None): 41635, ('e', None, None, None): 33665, (None, None, None, 's'): 24643, ('d', None, None, None): 23114, ('y', None, None, None): 20367, (None, None, None, 'p'): 19955, (None, None, None, 'c'): 18873, ('e', 'd', None, None): 17572, ('n', None, None, None): 17260, (None, None, None, 'a'): 15016, ...})

In [16]:
temp2 = [(elem, freq_2.get(elem)) for elem in freq_2]

So, we get the grams, then frequency distributions associated with each gram, and save them to self.

The word_len_dict takes each word in the data frame df and puts them in a big dictionary, all n letter words together by n

In [17]:
word_len_dict = {}
for i in range(3, 30):
    word_len_dict[i] = []
    for word in full_dict:
        if(len(word)>i):
            for j in range(len(word)-i+1):
                word_len_dict[i].append(word[j:j+i])

In [18]:
word_len_dict

{3: ['aaa',
  'aaa',
  'aaa',
  'aaa',
  'aaa',
  'aas',
  'aac',
  'ach',
  'che',
  'hen',
  'aae',
  'aee',
  'aah',
  'ahe',
  'hed',
  'aah',
  'ahs',
  'aal',
  'ale',
  'les',
  'esu',
  'sun',
  'und',
  'aal',
  'ali',
  'lii',
  'iis',
  'aal',
  'als',
  'lst',
  'aan',
  'and',
  'nda',
  'dah',
  'ahl',
  'aap',
  'aps',
  'pss',
  'aar',
  'ara',
  'rau',
  'aar',
  'ard',
  'rdv',
  'dva',
  'var',
  'ark',
  'aar',
  'ard',
  'rdw',
  'dwo',
  'wol',
  'olf',
  'aar',
  'are',
  'ren',
  'aar',
  'arg',
  'rgh',
  'aar',
  'ari',
  'rik',
  'ika',
  'aar',
  'aro',
  'ron',
  'oni',
  'nic',
  'aar',
  'aro',
  'ron',
  'oni',
  'nit',
  'ite',
  'aar',
  'aro',
  'ron',
  'ons',
  'nsb',
  'sbu',
  'bur',
  'urg',
  'aar',
  'arp',
  'aar',
  'arr',
  'rrg',
  'rgh',
  'ghh',
  'aas',
  'asv',
  'svo',
  'vog',
  'oge',
  'gel',
  'els',
  'aau',
  'aup',
  'aav',
  'avs',
  'vso',
  'aba',
  'bab',
  'abd',
  'bde',
  'deh',
  'aba',
  'bac',
  'aba',
  'bac',
  'aca'

candsort takes in the candidates, invalids, vowels and vowel ratio to guess

the readme said that the vowel ratio in words was at a certain level and the algorithm was guessing vowels too often, so they manually found the vowel ratio and forced the cantidate selection algorithm to throw out vowels past a point

Ok, now the big part of it... the ngram function... It is what matches the grams to the actual blanked out word.

It looks to me like there is no away around hard coding each possibility

In [19]:
test_word = "_pp__"
ans = "apple"

We have all the freqs, how many things match this test word?

In [20]:
freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)

freq_2 = [(item, freq_2.get(item)) for item in freq_2]
freq_3 = [(item, freq_3.get(item)) for item in freq_3]
freq_4 = [(item, freq_4.get(item)) for item in freq_4]
freq_5 = [(item, freq_5.get(item)) for item in freq_5]

In [21]:
freq_2[0][0]


('s', None)

In [22]:
def fd_to_string(fe):
    #Ok the elements in frequency distributions are in the form
    # (('e', 'n', 'e'), 3262)
    # tuple( tuple(string), int )
    letters = fe[0] #extract the letters
    #print(fe[0][0])
    #print(letters)
    output = ""
    for i in range(len(letters)):
        if letters[i] == None:
            output += "_"
        else:
            output += (letters[i])
    
    return output

In [23]:
fd_to_string(freq_2[0])

's_'

In [24]:
test = "s_"
print(re.search(test, "sandwich"))

None


In [25]:
freq_3[100]

(('e', 'n', 'e'), 3262)

In [26]:
fd_to_string(freq_3[100][0])

'e'

In [27]:
import fnmatch

In [28]:
import fnmatch

In [29]:
lst = ["_pp__",]
fnmatch.filter(lst, "p?")

[]

In [30]:
print(re.search("p_", "_pp__"))

<re.Match object; span=(2, 4), match='p_'>


In [31]:
bool(re.search("p.", ".pp.."))

True

In [32]:
bool(re.search("p.", "apple"))

True

In [33]:
print(freq_2[0])
print(freq_2[0][0])

(('s', None), 41635)
('s', None)


In [34]:
temp = []
_2gram = []
_3gram = []
_4gram = []    
_5gram = []

for word in full_dict:
    temp.append(list(word))         
    _2gram.extend(list(ngrams(word, 2)))
    _3gram.extend(list(ngrams(word, 3)))
    _4gram.extend(list(ngrams(word, 4)))
    _5gram.extend(list(ngrams(word, 5)))



from nltk import FreqDist
freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)
#print(_2gram[0])
freq_2 = [(item, freq_2.get(item)) for item in freq_2]
freq_3 = [(item, freq_3.get(item)) for item in freq_3]
freq_4 = [(item, freq_4.get(item)) for item in freq_4]
freq_5 = [(item, freq_5.get(item)) for item in freq_5]

In [50]:
temp = []
_2gram = []
_3gram = []
_4gram = []    
_5gram = []

for word in full_dict:
    temp.append(list(word))         
    _2gram.extend(list(ngrams(word, 2, pad_left=True, pad_right=True)))
    _3gram.extend(list(ngrams(word, 3, pad_left=True, pad_right=True)))
    _4gram.extend(list(ngrams(word, 4, pad_left=True, pad_right=True)))
    _5gram.extend(list(ngrams(word, 5, pad_left=True, pad_right=True)))



from nltk import FreqDist
freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)

freq_2 = [(item, freq_2.get(item)) for item in freq_2]
freq_3 = [(item, freq_3.get(item)) for item in freq_3]
freq_4 = [(item, freq_4.get(item)) for item in freq_4]
freq_5 = [(item, freq_5.get(item)) for item in freq_5]

In [152]:
def find_potential_guesses(freq_N, word, guess_list):
    #This takes as input a freq dist for n-grams and a word
    #it outputs all the n-grams from freq dist that match along with the 
    potential_guesses = []
    filtered_dist = []
    if(freq_N == []):
        dimension = 0
    else:
        dimension = len(freq_N[0][0])
    word_ngrams = list(ngrams(word, dimension))

    for fe in freq_N:
        #Ok..... so here is the idea...
        #if the ngram in fe matches, then it must be a match 
        #to one of the n-gons of the target word with wildcards
        fe_gram = fe[0]
        occurrences = fe[1]
        fe_gram = ["_" if x == None else x for x in fe_gram] #convert None to underscore if they exist
        #now, try to match it to one of the n-grams of the word...
        matched_letters_count = 0 #you must match at least one letter to be considered

        #possibilities, letters match, one is empty or both are empty
        #print(fe_gram[0])


        #for these guesses to be valid, they MUST guess something NOT IN THE GUESS LIST!
        #so, need to filter that out at the start.

        #im very tired and this is how im going to get rid of all the underscores
        fe_letters = []
        for l in fe_gram:
            if l != "_":
                fe_letters.append(l)
        #print(fe_letters)
        if fe_letters == []:
            is_subset = False
        else:
            is_subset = set(fe_letters).issubset(guess_list)
        
        #vowel_ratio = 

        ##also have to throw it out if guess is a vowel and ratio is too high?
        #vowels = list(("a", "e", "i", "o", "u"))
        #if fe_letters == []:
        #    vowelCondition = False
        #else:
        #    vowelCondition = set(fe_letters).issubset(vowels)
        vowelCondition = True #fix later

        

        if(is_subset == False and vowelCondition == True):
            for word_gram in word_ngrams:
                matched_letters_count = 0
                for i in range(dimension):
                    if(word_gram[i] != "_"): #there is a letter here, does the fe match?
                        if(fe_gram[i] == word_gram[i]):
                            matched_letters_count += 1
                        #elif(fe_gram[i] == "_"): #word has a letter and fe_gram has a blank so its fine
                        #    continue
                        else:
                            #this happens if we have a direct contradiction, immediately reject this
                            matched_letters_count = 0
                            break
                    #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                if(matched_letters_count != 0):
                    potential_guesses.append((set(fe_gram) - set(guess_list), dimension ,matched_letters_count,occurrences))
                    filtered_dist.append(fe)
                    #print(set(fe_gram).issubset(guess_list))
                    
    

    return potential_guesses, filtered_dist




        

In [144]:
def evaluate_potential_guesses(potential_guesses):
    #each potential guess is in the form:
    #({letters}, dimension from ngram, matched letters count, occurrences of pattern in dict)

    #So, which do we pick:
    #prioritize ngram length first
    #then pick based on most frequent letter
    #I guess how many letters match isnt as important...
    #I have read that vowel ratio may make a difference... maybe implement later

    sorted_guesses = sorted(potential_guesses, key = lambda tup: tup[1], reverse = True)
    if(sorted_guesses != []):
        max_dim = sorted_guesses[0][1]
        print(max_dim)
        max_dim_guesses = [x for x in sorted_guesses if x[1] == max_dim]
        max_dim_guesses.sort(key = lambda tup: tup[3], reverse = True)

        #oh what if theres multiple letters in the guesses thing...?
        max_guess = max_dim_guesses[0][0]
        if len(max_guess) > 1:
            max_guess = list(max_guess)[0]
        return max_guess
    else:
        return "!"

        


In [145]:
freq_2_guesses, freq_2_filtered = find_potential_guesses(freq_2,"_____", [])
freq_3_guesses, freq_3_filtered = find_potential_guesses(freq_3,"_____", [])
freq_4_guesses, freq_4_filtered = find_potential_guesses(freq_4,"_____", [])
freq_5_guesses, freq_5_filtered = find_potential_guesses(freq_5,"_____", [])
#print(freq_2_guesses[0:10])
#print(freq_3_guesses[0:10])
#print(freq_4_guesses[0:10])
#print(freq_5_guesses[0:10])

all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses
evaluate_potential_guesses(all_guesses)

'!'

In [138]:
freq_2_guesses, freq_2_filtered = find_potential_guesses(freq_2,"_pp_e", ["p", "e", "t"])
freq_3_guesses, freq_3_filtered = find_potential_guesses(freq_3,"_pp_e", ["p", "e", "t"])
freq_4_guesses, freq_4_filtered = find_potential_guesses(freq_4,"_pp_e", ["p", "e", "t"])
freq_5_guesses, freq_5_filtered = find_potential_guesses(freq_5,"_pp_e", ["p", "e", "t"])
#print(freq_2_guesses[0:10])
#print(freq_3_guesses[0:10])
#print(freq_4_guesses[0:10])
#print(freq_5_guesses[0:10])

all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses
evaluate_potential_guesses(all_guesses)


5


'r'

In [137]:
evaluate_potential_guesses(all_guesses)

5


'r'

IndexError: list index out of range

In [150]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list(["a", "e", "i", "o", "u"])
        self.guess_list = []


        #set up n-grams for current dictionary
        grams2 = []
        grams3 = []
        grams4 = []
        grams5 = []
        for word in self.full_dictionary:
            #is the word clean? It should be...
            #when I loaded the data it's fine...
            #the API has be a bit nervous so lets just put this here
            word = word.replace(" ", "") #remove any padding that may mess with this

            grams2.extend(list(ngrams(word, 2, pad_left = True, pad_right = True)))
            grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
            grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
            grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))
        
        #Ok, so now need the frequency distributions for these
        freq_dist_2 = FreqDist(grams2)
        freq_dist_3 = FreqDist(grams3)
        freq_dist_4 = FreqDist(grams4)
        freq_dist_5 = FreqDist(grams5)

        #need to actually extract the data from these distributions to use it
        self.freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
        self.freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
        self.freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
        self.freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

        self.words_by_length = {}
        min_word_length = 3 #shortest in the full_dict at least
        max_word_length = 40 #I guess this is pretty arbitrary

        for i in range(min_word_length, max_word_length):
            self.words_by_length.update({i : []})

        #I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
        #for word in self.full_dictionary:
        #    self.full_dictionary[len(word)].append(word)
        
            
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)


    #I think this is an easier way to apply ngrams... I don't really know as much about the subject
        #so there is an equal or greater likelihood that this is completely wrong
    def find_potential_guesses(freq_N, word, guess_list):
        #This takes as input a freq dist for n-grams and a word
        #it outputs all the n-grams from freq dist that match along with the 
        potential_guesses = []
        filtered_dist = []
        if(freq_N == []):
            dimension = 0
        else:
            dimension = len(freq_N[0][0])
        word_ngrams = list(ngrams(word, dimension))

        for fe in freq_N:
            #Ok..... so here is the idea...
            #if the ngram in fe matches, then it must be a match 
            #to one of the n-gons of the target word with wildcards
            fe_gram = fe[0]
            occurrences = fe[1]
            fe_gram = ["_" if x == None else x for x in fe_gram] #convert None to underscore if they exist
            #now, try to match it to one of the n-grams of the word...
            matched_letters_count = 0 #you must match at least one letter to be considered

            #possibilities, letters match, one is empty or both are empty
            #print(fe_gram[0])


            #for these guesses to be valid, they MUST guess something NOT IN THE GUESS LIST!
            #so, need to filter that out at the start.

            #im very tired and this is how im going to get rid of all the underscores
            fe_letters = []
            for l in fe_gram:
                if l != "_":
                    fe_letters.append(l)
            #print(fe_letters)
            if fe_letters == []:
                is_subset = False
            else:
                is_subset = set(fe_letters).issubset(guess_list)
            
            #vowel_ratio = 

            ##also have to throw it out if guess is a vowel and ratio is too high?
            #vowels = list(("a", "e", "i", "o", "u"))
            #if fe_letters == []:
            #    vowelCondition = False
            #else:
            #    vowelCondition = set(fe_letters).issubset(vowels)
            vowelCondition = True #fix later

            

            if(is_subset == False and vowelCondition == True):
                for word_gram in word_ngrams:
                    matched_letters_count = 0
                    for i in range(dimension):
                        if(word_gram[i] != "_"): #there is a letter here, does the fe match?
                            if(fe_gram[i] == word_gram[i]):
                                matched_letters_count += 1
                            #elif(fe_gram[i] == "_"): #word has a letter and fe_gram has a blank so its fine
                            #    continue
                            else:
                                #this happens if we have a direct contradiction, immediately reject this
                                matched_letters_count = 0
                                break
                        #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                    if(matched_letters_count != 0):
                        potential_guesses.append((set(fe_gram) - set(guess_list), dimension ,matched_letters_count,occurrences))
                        filtered_dist.append(fe)
                        #print(set(fe_gram).issubset(guess_list))
                        
        

        return potential_guesses, filtered_dist

        
    def evaluate_potential_guesses(potential_guesses):
        #each potential guess is in the form:
        #({letters}, dimension from ngram, matched letters count, occurrences of pattern in dict)

        #So, which do we pick:
        #prioritize ngram length first
        #then pick based on most frequent letter
        #I guess how many letters match isnt as important...
        #I have read that vowel ratio may make a difference... maybe implement later

        sorted_guesses = sorted(potential_guesses, key = lambda tup: tup[1], reverse = True)
        max_dim = sorted_guesses[0][1]
        print(max_dim)
        max_dim_guesses = [x for x in sorted_guesses if x[1] == max_dim]
        max_dim_guesses.sort(key = lambda tup: tup[3], reverse = True)

        #oh what if theres multiple letters in the guesses thing...?
        #just pick one?
        max_guess = max_dim_guesses[0][0]
        if len(max_guess) > 1:
            max_guess = list(max_guess)[0]
        return max_guess
    


    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        #clean_word = word.replace("_",".") #I am using underscores

        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        #print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'

        freq_2_guesses, freq_dist_2_filtered = find_potential_guesses(self.freq_dist_2, word, self.guess_list)
        freq_3_guesses, freq_dist_3_filtered = find_potential_guesses(self.freq_dist_3, word, self.guess_list)
        freq_4_guesses, freq_dist_4_filtered = find_potential_guesses(self.freq_dist_4, word, self.guess_list)
        freq_5_guesses, freq_dist_5_filtered = find_potential_guesses(self.freq_dist_5, word, self.guess_list)

        #update freq_dists
        self.freq_dist_2 = freq_dist_2_filtered
        self.freq_dist_3 = freq_dist_3_filtered
        self.freq_dist_4 = freq_dist_4_filtered
        self.freq_dist_5 = freq_dist_5_filtered


        all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses
        max_guess = evaluate_potential_guesses(all_guesses)
        
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        

        if(max_guess != "!"):
            guess_letter = max_guess
        



        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                print("Success! the word was: %s" % word)
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            print("You Lose, the answer was: %s" % self.correct_word)
                



In [151]:
game = MyHangman()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: ______.
Guessing letter: e
Incorrect guess e, # of tries remaining: 6. Word: ______.


IndexError: list index out of range